In [0]:
from google.colab import files
files.upload()

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import json






dataset = pd.read_csv('dataset.tsv', delimiter = '\t', quoting = 3,error_bad_lines=False)
print(dataset)

corpus = []
for i in range(0, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Title'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

cv = CountVectorizer(max_features=15000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 3].values


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.40)
#print(X_train,X_test,y_train,y_test)

y_train2=list(set(y_train))
allwords={}
for i in range(len(y_train2)):
  allwords[i]=y_train2[i]
  allwords[y_train2[i]]=i
  
#print(allwords)

X_train1=[[int(X_train[i][j]) for j in range(len(X_train[i]))] for i in range(len(X_train))]
y_train1=[allwords[i] for i in list(y_train)]
for i in range(len(y_train1)):
  index=y_train1[i];
  y_train1[i]=[0 for j in range(13)]
  y_train1[i][index]=1
  
X_test1=[[int(X_test[i][j]) for j in range(len(X_test[i]))] for i in range(len(X_test))]
y_test1=[allwords[i] for i in list(y_test)]
for i in range(len(y_test1)):
  index=y_test1[i];
  y_test1[i]=[0 for j in range(13)]
  y_test1[i][index]=1

print(y_train1)

data={"X_train":X_train1,"X_test":X_test1,"y_train":y_train1,"y_test":y_test1,"allwords":allwords}
print(data)


In [0]:
with open("data1.json","w") as filename:
    json.dump(data,filename)


In [0]:
from keras import Sequential
from keras.layers import Dense
import numpy as np
from random import random

no_of_inputs=len(data["X_train"][0])
no_of_output=len(data["y_train"][0])


#no_of_inputs=547 and o_of_outputs=13 for the dataset

model=Sequential()
model.add(Dense(50,input_shape=(no_of_inputs,),activation="relu"))
model.add(Dense(20,activation="relu"))
model.add(Dense(no_of_output,activation="softmax"))

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.array(data["X_train"]), np.array(data["y_train"]), epochs=50, batch_size=10)

#Getting the weights of the trained neural network
weights=model.get_weights()

prediction=model.predict(np.array(data["X_test"]))
count=0
for i in range(len(prediction)):
  #print(prediction[i])
  #print(np.argmax(prediction[i]),np.argmax(data["y_test"][i]))
  if np.argmax(prediction[i])==np.argmax(data["y_test"][i]):
    count+=1
print("Accuracy is:-",(count/len(prediction))*100)


In [0]:
print(weights)
print(allwords)